# TTP

In [51]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [52]:
%pip install openpyxl

df = pd.read_excel('../data/processed/DB People.xlsx', sheet_name='DB')

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [53]:
#Visualizar que la base de datos queda bien cargada
df.head()

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada


In [54]:
# ...existing code...

# Define los horarios de referencia
horarios = [pd.to_datetime('00:00').time(), pd.to_datetime('08:00').time(), pd.to_datetime('16:00').time()]

def horario_marcado(row):
    # Combina fecha y hora para obtener un datetime completo
    fecha = row['Fecha'] if 'Fecha' in row else pd.Timestamp.today().date()
    hora = row['Hora']
    # Convierte 'hora' a objeto time si es string
    if isinstance(hora, str):
        hora = pd.to_datetime(hora).time()
    dt = pd.Timestamp.combine(fecha, hora)
    tipo = row['Tipo Acceso'].lower()
    for h in horarios:
        ref = pd.Timestamp.combine(fecha, h)
        if tipo == 'entrada':
            if ref - pd.Timedelta(hours=6) <= dt <= ref + pd.Timedelta(hours=2):
                return h.strftime('%H:%M')
        elif tipo == 'salida':
            if ref - pd.Timedelta(hours=4) <= dt <= ref + pd.Timedelta(hours=4):
                return h.strftime('%H:%M')
    return np.nan

df['Horario Entrada'] = df.apply(lambda row: horario_marcado(row) if row['Tipo Acceso'].lower() == 'entrada' else np.nan, axis=1)
df['Horario Salida'] = df.apply(lambda row: horario_marcado(row) if row['Tipo Acceso'].lower() == 'salida' else np.nan, axis=1)

# Visualiza el resultado
df[['Fecha', 'Hora', 'Tipo Acceso', 'Horario Entrada', 'Horario Salida']]
# ...existing code...

,Fecha,Hora,Tipo Acceso,Horario Entrada,Horario Salida
0,2025-06-03,07:36:00,Entrada,08:00,NaN
1,2025-06-03,17:17:00,Salida,NaN,16:00
2,2025-06-04,05:02:00,Entrada,08:00,NaN
3,2025-06-04,06:02:00,Salida,NaN,08:00
4,2025-06-04,09:58:00,Entrada,08:00,NaN
...,...,...,...,...,...
4747,2025-06-11,17:18:00,Salida,NaN,16:00
4748,2025-06-12,07:40:00,Entrada,08:00,NaN
4749,2025-06-12,14:11:00,Salida,NaN,16:00
4750,2025-06-12,14:14:00,Entrada,16:00,NaN


In [55]:
#Ahora quiero junta la columna Horario Entrada y Salida
df['Horario Combinado'] = df.apply(lambda row: f"{row['Horario Entrada']} - {row['Horario Salida']}" if pd.notna(row['Horario Entrada']) and pd.notna(row['Horario Salida']) else (row['Horario Entrada'] if pd.notna(row['Horario Entrada']) else row['Horario Salida']), axis=1)
df

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,Horario Entrada,Horario Salida,Horario Combinado
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,08:00,08:00
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-11,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4748,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,07:40:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
4749,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:11:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4750,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:14:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,16:00,NaN,16:00


In [56]:
#Ahora quiero guardar un CSV con el df nuevo
df.to_csv('../data/processed/DB People con Horarios.csv', index=False)

In [57]:


df["Fecha"] = pd.to_datetime(df["Fecha"]).dt.date
df = df.sort_values(by=["Nombre", "Fecha", "Hora"]).reset_index(drop=True)
def balancear_registros(grupo):
    registros = []
    esperado = "Entrada"  # siempre arranca con Entrada

    for _, row in grupo.iterrows():
        actual = row["Tipo Acceso"]

        # Mientras el actual no coincida con lo esperado, inserta faltantes
        while actual != esperado:
            faltante = row.copy()
            faltante["Tipo Acceso"] = esperado + " (Faltante)"
            faltante["Hora"] = pd.NaT
            registros.append(faltante)
            # alternar esperado
            esperado = "Salida" if esperado == "Entrada" else "Entrada"

        # Ahora sí agregamos el registro real
        registros.append(row)
        esperado = "Salida" if esperado == "Entrada" else "Entrada"

    return pd.DataFrame(registros)

# Aplicar por persona y fecha
df_completo = df.groupby(["Nombre", "Fecha"], group_keys=False).apply(balancear_registros)

df_completo = df_completo.reset_index(drop=True)
print(df_completo[["Nombre", "Fecha", "Hora", "Tipo Acceso"]])



                            Nombre       Fecha      Hora         Tipo Acceso
0     ADRIANA LUCIA CABRA QUINTERO  2025-06-09  06:07:00             Entrada
1     ADRIANA LUCIA CABRA QUINTERO  2025-06-09       NaT   Salida (Faltante)
2     ADRIANA LUCIA CABRA QUINTERO  2025-06-09  06:08:00             Entrada
3     ADRIANA LUCIA CABRA QUINTERO  2025-06-09  17:17:00              Salida
4     ADRIANA LUCIA CABRA QUINTERO  2025-06-10  07:36:00             Entrada
...                            ...         ...       ...                 ...
5937     YONATAN SALAMANCA SANTANA  2025-06-05  07:49:00             Entrada
5938     YONATAN SALAMANCA SANTANA  2025-06-05  16:04:00              Salida
5939     YONATAN SALAMANCA SANTANA  2025-06-05  23:41:00             Entrada
5940     YONATAN SALAMANCA SANTANA  2025-06-06       NaT  Entrada (Faltante)
5941     YONATAN SALAMANCA SANTANA  2025-06-06  08:08:00              Salida

[5942 rows x 4 columns]


/tmp/ipykernel_1144/2090427555.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_completo = df.groupby(["Nombre", "Fecha"], group_keys=False).apply(balancear_registros)


In [58]:
df_completo

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,Horario Entrada,Horario Salida,Horario Combinado
0,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,06:07:00,Tibasosa Izq Entrada Funcionarios,Asistencia,Entrada,08:00,NaN,08:00
1,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,NaT,Tibasosa Der Entrada Contratistas,Asistencia,Salida (Faltante),08:00,NaN,08:00
2,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,06:08:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
3,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-10,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5937,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,07:49:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
5938,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,16:04:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
5939,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,23:41:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,NaN,NaN,NaN
5940,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-06,NaT,Tibasosa Izq Salida Funcionarios,Asistencia,Entrada (Faltante),NaN,08:00,08:00


In [59]:


def reducir_a_dos(grupo):
    grupo = grupo.sort_values(by="Horario Combinado").reset_index(drop=True)

    # Buscar primera Entrada (real o faltante)
    entrada = grupo[grupo["Tipo Acceso"].str.contains("Entrada")].head(1)

    # Buscar última Salida (real o faltante)
    salida = grupo[grupo["Tipo Acceso"].str.contains("Salida")].tail(1)

    # Concatenar resultados (si no existe entrada o salida, quedará solo una fila)
    return pd.concat([entrada, salida])

# Aplicar por persona y fecha
df_final = df_completo.groupby(["Nombre", "Fecha"], group_keys=False).apply(reducir_a_dos)
df_final = df_final.reset_index(drop=True)




/tmp/ipykernel_1144/1367529354.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_completo.groupby(["Nombre", "Fecha"], group_keys=False).apply(reducir_a_dos)


In [60]:
df_final

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,Horario Entrada,Horario Salida,Horario Combinado
0,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,06:07:00,Tibasosa Izq Entrada Funcionarios,Asistencia,Entrada,08:00,NaN,08:00
1,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
2,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-10,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
3,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-10,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-11,NaT,Tibasosa Izq Salida Funcionarios,Asistencia,Entrada (Faltante),NaN,16:00,16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4683,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-04,08:07:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,08:00,08:00
4684,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,07:49:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
4685,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,16:04:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4686,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-06,NaT,Tibasosa Izq Salida Funcionarios,Asistencia,Entrada (Faltante),NaN,08:00,08:00


In [61]:

def ajustar_faltantes(grupo):
    grupo = grupo.sort_values(by="Horario Combinado").reset_index(drop=True)

    # Convertir Horario Combinado a datetime si no lo es
    grupo["Horario Combinado"] = pd.to_datetime(grupo["Horario Combinado"], format="%H:%M", errors="coerce")

    for i, row in grupo.iterrows():
        # Si es salida faltante → tomar anterior + 8h
        if "Salida (Faltante)" in row["Tipo Acceso"]:
            if i > 0 and pd.notna(grupo.at[i-1, "Horario Combinado"]):
                grupo.at[i, "Horario Combinado"] = grupo.at[i-1, "Horario Combinado"] + pd.Timedelta(hours=8)

        # Si es entrada faltante → tomar siguiente - 8h
        if "Entrada (Faltante)" in row["Tipo Acceso"]:
            if i < len(grupo)-1 and pd.notna(grupo.at[i+1, "Horario Combinado"]):
                grupo.at[i, "Horario Combinado"] = grupo.at[i+1, "Horario Combinado"] - pd.Timedelta(hours=8)

    # Opcional: volver a formatear a string "HH:MM" si lo necesitas
    grupo["Horario Combinado"] = grupo["Horario Combinado"].dt.strftime("%H:%M")

    return grupo

# Aplicar por persona y fecha
df_final = df_final.groupby(["Nombre", "Fecha"], group_keys=False).apply(ajustar_faltantes).reset_index(drop=True)




/tmp/ipykernel_1144/3017762009.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_final.groupby(["Nombre", "Fecha"], group_keys=False).apply(ajustar_faltantes).reset_index(drop=True)


In [62]:
df_final

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,Horario Entrada,Horario Salida,Horario Combinado
0,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,06:07:00,Tibasosa Izq Entrada Funcionarios,Asistencia,Entrada,08:00,NaN,08:00
1,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-09,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
2,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-10,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
3,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-10,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4,Calidad,38612552,ADRIANA LUCIA CABRA QUINTERO,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-11,NaT,Tibasosa Izq Salida Funcionarios,Asistencia,Entrada (Faltante),NaN,16:00,08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4683,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-04,08:07:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,08:00,08:00
4684,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,07:49:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
4685,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-05,16:04:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4686,Ingenieria,38612335,YONATAN SALAMANCA SANTANA,Bavaria S.A.,MAINTENANCE AND SERVICES COORDINATION,2025-06-06,NaT,Tibasosa Izq Salida Funcionarios,Asistencia,Entrada (Faltante),NaN,08:00,00:00


In [ ]:
#Ahora voy a eliminar columna Horario Entrada y Horario Salida
df_final = df_final.drop(columns=["Horario Entrada", "Horario Salida"])
df_final


In [64]:
# Reparar filas faltantes sin hora
for i in range(len(df)):
    tipo = str(df.loc[i, "Tipo Acceso"])
    
    # Caso: Salida (Faltante) sin hora
    if "Salida" in tipo and "Faltante" in tipo and pd.isna(df.loc[i, "Horario Combinado"]):
        if i > 0 and pd.notna(df.loc[i-1, "Horario Combinado"]):
            df.loc[i, "Horario Combinado"] = df.loc[i-1, "Horario Combinado"] + pd.Timedelta(hours=8)

    # Caso: Entrada (Faltante) sin hora
    elif "Entrada" in tipo and "Faltante" in tipo and pd.isna(df.loc[i, "Horario Combinado"]):
        if i < len(df)-1 and pd.notna(df.loc[i+1, "Horario Combinado"]):
            df.loc[i, "Horario Combinado"] = df.loc[i+1, "Horario Combinado"] - pd.Timedelta(hours=8)
# Guardar a Excel
df_final.to_excel("../data/processed/DB People_resumido_ajustado.xlsx", index=False)